### Ingestion del archivo "genre.csv"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Paso 1 - Leer el archivo CSV usando "DataFraome" de Spark

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [0]:
genre_schema = StructType( fields=[
    StructField("genreId", IntegerType(), False),
    StructField("genreName", StringType(), True)   
])

In [0]:
#Agregamos v_file_date, ya que ahora en nuestro Data Lake, tenemos carpetas con la fecha
#Seccion carga incormental, para esta carga por ser pocos datos, sera una carga completa
genre_df = spark.read \
            .option("header", True) \
            .schema(genre_schema) \
            .csv(f"{bronze_folder_path}/{v_file_date}/genre.csv")

%md
### Paso 2 - Cambiar el nombre a las columnas" según lo requerido"

In [0]:
genre_renamed_df = genre_df.withColumnsRenamed({"genreId":"genre_id", "genreName":"genre_name"})

### Paso 3 -Agregar la columna "ingestion_date" y "enviroment" al DataFrame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
genre_final_df = add_ingestion_date(genre_renamed_df) \
                 .withColumn("environment", lit(v_environment)) \
                 .withColumn("file_date", lit(v_file_date))

### Paso 4 - Escribir datos en el DataLake en formato Parquet

In [0]:
# genre_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/genres")

# Crear una tabla en base al data frame genre_final_df
# Creara la tabla genres en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver
#genre_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.genres")

#Modulo: Delta Lake
genre_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.genres")

In [0]:
%sql
-- Consultamos la DB movie_silver la tabla genres
SELECT * FROM movie_silver.genres;

In [0]:
dbutils.notebook.exit("Success")